In [1]:
import sys
print(sys.executable)

c:\Users\essai\AppData\Local\Programs\Python\Python313\python.exe


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt


### Variables floues

In [3]:
Distance_to_next_car = ctrl.Antecedent(np.arange(0, 101, 1), 'Distance_to_next_car')
Speed_change = ctrl.Antecedent(np.arange(-20, 21, 1), 'Speed_change')
Acceleration_Adjust = ctrl.Consequent(np.arange(-10, 11, 1), 'Acceleration_Adjust')

### Fonctions d’appartenance

In [4]:
Distance_to_next_car['close'] = fuzz.trapmf(Distance_to_next_car.universe, [0, 0, 10, 40])
Distance_to_next_car['medium'] = fuzz.trimf(Distance_to_next_car.universe, [20, 50, 80])
Distance_to_next_car['far'] = fuzz.trapmf(Distance_to_next_car.universe, [60, 90, 100, 100])

Speed_change['declining'] = fuzz.trapmf(Speed_change.universe, [-20, -20, -10, 0])
Speed_change['constant'] = fuzz.trimf(Speed_change.universe, [-5, 0, 5])
Speed_change['growing'] = fuzz.trapmf(Speed_change.universe, [0, 10, 20, 20])

Acceleration_Adjust['decrease'] = fuzz.trapmf(Acceleration_Adjust.universe, [-10, -10, -5, 0])
Acceleration_Adjust['stable'] = fuzz.trimf(Acceleration_Adjust.universe, [-2, 0, 2])
Acceleration_Adjust['increase'] = fuzz.trapmf(Acceleration_Adjust.universe, [0, 5, 10, 10])

### Règles floues

In [5]:
rule1 = ctrl.Rule(Distance_to_next_car['close'] & Speed_change['growing'], Acceleration_Adjust['decrease'])
rule2 = ctrl.Rule(Distance_to_next_car['far'] & Speed_change['declining'], Acceleration_Adjust['increase'])
rule3 = ctrl.Rule(Speed_change['constant'], Acceleration_Adjust['stable'])

### Système de contrôle

In [6]:
acc_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
acc_sim = ctrl.ControlSystemSimulation(acc_ctrl)

### Génération du dataset (1000 observations)

In [8]:
np.random.seed(42)
n = 1000
data = []

for _ in range(n):
    dist = np.random.uniform(0, 100)
    spd = np.random.uniform(-20, 20)
    # 👉 créer une nouvelle simulation à chaque itération
    acc_sim = ctrl.ControlSystemSimulation(acc_ctrl)
    acc_sim.input['Distance_to_next_car'] = dist
    acc_sim.input['Speed_change'] = spd
    try:
        acc_sim.compute()
        acc = acc_sim.output['Acceleration_Adjust']
    except KeyError:
        acc = 0  # Valeur par défaut si aucune règle activée
    data.append([dist, spd, acc])

data = np.array(data)
print("✅ Dataset généré avec succès :", data.shape)

✅ Dataset généré avec succès : (1000, 3)


### Entraînement du modèle ML

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

X = data[:, :2]   # entrées : distance, speed_change
y = data[:, 2]    # sortie floue défuzzifiée

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


### Évaluation des performances

In [10]:
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)

print(f"Erreur absolue moyenne (MAE) : {MAE:.3f}")
print(f"Erreur quadratique moyenne (MSE) : {MSE:.3f}")


Erreur absolue moyenne (MAE) : 0.040
Erreur quadratique moyenne (MSE) : 0.024
